In [16]:
import torch
import os
from torchvision import datasets, models, transforms
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
import neptune
#import splitfolders # Библиотека для разделения файлов картинок на train test
import matplotlib.gridspec as gridspec
cudnn.benchmark = True

In [17]:
run = neptune.init_run(
    project="neas1231/Neas1231",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmY2VjMGIzOS01ZjI1LTQ1MTItODQxYi0zMjIyOWIwYWI0MzIifQ==",
)

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256 , 256)),   
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256 , 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\dataset'
run["config/data_dir"] = data_dir

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
              for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) 
              for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

https://app.neptune.ai/neas1231/Neas1231/e/NEAS-76


In [18]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=14):
    since = time.time()
    #Сохраняем  лучшие веса 
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 99
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)


        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            # перенос  вычислений на куду 
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                
                optimizer.zero_grad()

                #
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    acc = (torch.sum(preds == labels.data)) / len(inputs)
                    run["logs/training/batch/loss"].append(loss)
                    run["logs/training/batch/acc"].append(acc)

                    # считаем градиенты тольок если трейн данные 
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # стасистика  
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # сохраняем не только лучшие веса , но и акураси 
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Loss: {best_loss:4f}')
    

    model.load_state_dict(best_model_wts)
    return model

In [19]:
model_ft = models.resnet152(pretrained=True) 
num_ftrs = model_ft.fc.in_features
# num_ftrs = model_ft.fc.in_features Для рес нета , но у эфинета другая архитектура 

model_ft.fc = nn.Linear(num_ftrs, len(class_names))
# cuda
model_ft = model_ft.to(device)
run["config/model"] = model_ft

criterion = nn.CrossEntropyLoss()
run["config/criterion"] = criterion

# оптимайзер  с лернинг рейт 
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
run["config/optimizer"] = optimizer_ft
run["parameters"] = model_ft.parameters()

# редактируем лернинг рейт каждые 7 шагов 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


In [20]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)
torch.save(model_ft,'resnet152-bestloss.pth')
torch.save(model_ft.state_dict(), "weights-resnet152-bestloss.pth")
run.stop()

Epoch 0/19
----------
train Loss: 1.3929 Acc: 0.4997
val Loss: 1.0922 Acc: 0.6297

Epoch 1/19
----------
train Loss: 0.9878 Acc: 0.6543
val Loss: 0.9502 Acc: 0.6805

Epoch 2/19
----------
train Loss: 0.8559 Acc: 0.6973
val Loss: 0.8090 Acc: 0.7239

Epoch 3/19
----------
train Loss: 0.7939 Acc: 0.7254
val Loss: 0.6437 Acc: 0.7793

Epoch 4/19
----------
train Loss: 0.7162 Acc: 0.7533
val Loss: 0.6107 Acc: 0.7858

Epoch 5/19
----------
train Loss: 0.6652 Acc: 0.7686
val Loss: 0.5732 Acc: 0.8163

Epoch 6/19
----------
train Loss: 0.6455 Acc: 0.7751
val Loss: 0.6118 Acc: 0.7959

Epoch 7/19
----------
train Loss: 0.5020 Acc: 0.8268
val Loss: 0.4038 Acc: 0.8532

Epoch 8/19
----------
train Loss: 0.4431 Acc: 0.8490
val Loss: 0.3884 Acc: 0.8661

Epoch 9/19
----------
train Loss: 0.4183 Acc: 0.8554
val Loss: 0.3824 Acc: 0.8569

Epoch 10/19
----------
train Loss: 0.4063 Acc: 0.8584
val Loss: 0.3585 Acc: 0.8717

Epoch 11/19
----------
train Loss: 0.3849 Acc: 0.8659
val Loss: 0.3592 Acc: 0.8744

Ep

In [15]:
import gc
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)
gc.collect()

10189

In [10]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:8192"

In [7]:
run = neptune.init_run(
    project="neas1231/Neas1231",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmY2VjMGIzOS01ZjI1LTQ1MTItODQxYi0zMjIyOWIwYWI0MzIifQ==",
)
run["config/data_dir"] = data_dir
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.Adam(model_conv.fc.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

https://app.neptune.ai/neas1231/Neas1231/e/NEAS-69
